In [ ]:
using Pkg; Pkg.activate(".")
using SteinMD
using Molly 
using PotentialLearning
using LinearAlgebra

In [ ]:
include("/Users/swyant/cesmix/dev/SteinMD.jl/src/makie/makie.jl")

In [ ]:
ref = MullerBrown()

In [ ]:
# define properties
atom_mass = 1.0u"g/mol"
boundary = RectangularBoundary(Inf*u"nm")
temp = 100.0u"K"

# define initial system
#atoms = [Atom(mass=atom_mass, σ=0.3u"nm", ϵ=0.2u"kJ * mol^-1")] # Do you need to specify the LJ params?
atoms = [Atom(mass=atom_mass)] # No, I don't think so
coords = [SVector(-0.8, 1.2)u"nm"] # initial position
sys = System(
    atoms=atoms,
    coords=coords,
    boundary=boundary,
    general_inters=(ref,),
    loggers=(coords=CoordinateLogger(100; dims=2),),
)

# define simulator
sim_langevin = OverdampedLangevin(
            dt=0.002u"ps",
            temperature=temp,
            friction=4.0u"ps^-1")

In [ ]:

# run simulation - this will take a few seconds
simulate!(sys, sim_langevin, 5_000_000)

In [ ]:
# plot MD trajectory 
dist_units = u"nm"
xcoords = Vector(-3.5:0.05:1.5) .* dist_units
ycoords = Vector(-1.5:0.05:3.5) .* dist_units
ctr_grid = [xcoords, ycoords]
ctr_lvls = -150:20:400

f0 = plot_md_trajectory(sys, ctr_grid, fill=true, lvls=ctr_lvls)

In [ ]:
# define kernel
#rbf = RBF(Euclidean(2), β=1.0, ℓ = 0.1) 
rbf = RBF(Euclidean(2), β=1.0, ℓ = 1.0) 
# define simulator
sim_svgd = StochasticSVGD(
            dt=0.002u"ps",
            kernel=rbf,
            #temperature=temp, # 1.0u"K",
            temperature=200.0u"K", 
            friction=1.0u"ps^-1")

In [ ]:
# subsample to obtain training data
coords_train = [sys.loggers.coords.history[i][1] for i = 2:2000:length(sys.loggers.coords.history)]
ntrain = length(coords_train)
atoms_train = [Atom(mass=atom_mass) for i in 1:ntrain]

# define initial ensemble
ens1 = [System(
    atoms=[atoms_i],
    coords=[coords_i],
    boundary=boundary,
    general_inters=(ref,),
    # k = 1.0u"kJ * K^-1 * mol^-1",
    loggers=(
        coords=CoordinateLogger(1; dims=2),
        ksd=StepComponentLogger(1; dims=2),
    )
) for (atoms_i, coords_i) in zip(atoms_train, coords_train)]

In [ ]:
# run simulation - this will take a few seconds
simulate!(ens1, sim_svgd, 10_000) # Note number of simulations chosen to lead to roughly equivalent sim run times as running the full trajectory

In [ ]:
f1 = plot_md_trajectory(ens1, ctr_grid, fill=true, lvls=ctr_lvls, showpath=false)

In [ ]:

f1b, _ = plot_step(ens1, "Kernel Stein discrepancy", logscl=true)
f1b

In [ ]:
# subsample to obtain training data
x_train = [[1.0, 0.0], [0.5, 0.0], [0.0, 0.0], [-0.5, 0.0], [-1.0, 0.0],
           [1.0, 0.5], [0.5, 0.5], [0.0, 0.5], [-0.5, 0.5], [-1.0, 0.5],
           [0.5, 1.0], [0.0, 1.0], [-0.5, 1.0], [-1.0, 1.0], [-1.5, 1.0],
           [0.5, 1.5], [0.0, 1.5], [-0.5, 1.5], [-1.0, 1.5], [-1.5, 1.5], 
           [0.0, 2.0], [-0.5, 2.0], [-1.0, 2.0], [-1.5, 2.0], [-2.0, 2.0]]
coords_train = [SVector(x[1], x[2])u"nm" for x in x_train]
ntrain = length(x_train)
atoms_train = [Atom(mass=atom_mass, σ=1.0u"nm", ϵ=1.0u"kJ * mol^-1") for i in 1:ntrain]

# define initial ensemble
ens2 = [System(
    atoms=[atoms_i],
    coords=[coords_i],
    boundary=boundary,
    general_inters=(ref,),
    # k = 1.0u"kJ * K^-1 * mol^-1",
    loggers=(
        coords=CoordinateLogger(1; dims=2),
        ksd=StepComponentLogger(1; dims=2),
    )
) for (atoms_i, coords_i) in zip(atoms_train, coords_train)]

In [ ]:
# run simulation - this will take a few seconds
simulate!(ens2, sim_svgd, 10_000)

In [ ]:
f2 = plot_md_trajectory(ens2, ctr_grid, fill=true, lvls=ctr_lvls, showpath=false)


In [ ]:
f2b, _ = plot_step(ens2, "Kernel Stein discrepancy", logscl=true)
f2b